In [10]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [11]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [13]:
import cookin_ink

reload(cookin_ink)


DEFAULT_PARAMS = {
    'ink_lookback_window': 10,            # Number of periods for momentum lookback
    'ink_momentum_threshold': 0.008,      # Change in return required to trigger a trade
    'ink_past_lag': 1,                     # Number of periods to look back for past return
    'ink_clear_threshold': 0.0001,         # Threshold below which we clear the position,
}

param_grid = {
    'ink_lookback_window': [5, 8, 10, 15, 20, 25, 30],
    'ink_momentum_threshold': [0.005, 0.007, 0.008, 0.009, 0.01, 0.012],
    'ink_past_lag': [1, 2, 3],
    'ink_clear_threshold': [0.0001],
}

param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = cookin_ink.Trader(params)
    # days = [1, 2, 3]
    ogd = [0, 2]
    # days = [1, 3]
    for rnum in [1, 4]:
        start_day = -2 if rnum == 1 else 1
        days = [start_day + d for d in ogd]
        profits = backtest_days(trader, days=days, data_path='.', round_num=rnum)
        for d in days:
            params[f'profit_round_{rnum}_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 126/126 [13:56<00:00,  6.64s/it]


In [22]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[col for col in df.columns if col.startswith('profit_')]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df.head(10)

,ink_lookback_window,ink_momentum_threshold,ink_past_lag,ink_clear_threshold,profit_round_1_day_-2,profit_round_1_day_0,profit_round_4_day_1,profit_round_4_day_3,tot_profit
36,10,0.005,1,0.0001,8342.0,-3442.5,746.0,339.0,5984.5
24,8,0.008,1,0.0001,3310.0,0.0,1562.0,-66.5,4805.5
30,8,0.010,1,0.0001,3270.0,0.0,1562.0,-66.5,4765.5
27,8,0.009,1,0.0001,3270.0,0.0,1562.0,-66.5,4765.5
21,8,0.007,1,0.0001,1635.0,0.0,1562.0,1178.5,4375.5
33,8,0.012,1,0.0001,2430.0,0.0,1562.0,-66.5,3925.5
14,5,0.010,3,0.0001,2024.0,246.5,2107.0,-501.0,3876.5
42,10,0.008,1,0.0001,1393.0,83.5,1533.0,166.0,3175.5
51,10,0.012,1,0.0001,1836.0,0.0,1302.0,-112.0,3026.0
48,10,0.010,1,0.0001,1836.0,0.0,1302.0,-112.0,3026.0


In [ ]:
profit_cols = [col for col in df.columns if col.startswith('profit_')]
# df[(df[profit_cols] > 0).all(axis=1)]
profit_cols

,ink_lookback_window,ink_momentum_threshold,ink_past_lag,ink_clear_threshold,profit_round_1_day_-2,profit_round_1_day_0,profit_round_4_day_1,profit_round_4_day_3,tot_profit
42,10,0.008,1,0.0001,1393.0,83.5,1533.0,166.0,3175.5


In [7]:
df.to_csv('backtests/ink_trader_momentum_day1.csv')